# Task 4: Demonstrating Quantum Advantage

In this task we demonstrate how introducing a bias into a random number generator suddenly makes it possible for a ML based predictor to start predicting the next bits with reasonable accuracy.

To do the same, we first attempt to -
+ Run the predictor for random numbers generated from a Jitter RNG in a manner identical to what was done in Task 2.
+ Run the same predictor after biasing all the bits to a small degree.

In [13]:
from random import random

In [14]:
import hashlib
from collections import deque
import math
import time


class JitterRNG:
    def __init__(self, pool_size=256):
        self.entropy_pool = deque(maxlen=pool_size)
        self.pool_size = pool_size
        self.last_time = None

    def _collect_timing_jitter(self, iterations=1000):
        """
        Collect entropy from timing variations between CPU operations
        """
        jitter_data = []

        self.last_time = time.perf_counter_ns()

        for _ in range(iterations):
            # We need to introduce variability somehow.
            # In the following space, do some arbitrary computation to introduce variability.
            # We're not going to judge this part. Just make it reasonable so your notebook
            # doesn't take forever to run.

            _ = math.sqrt(math.pow(math.pi, math.e) / math.pow(math.e, math.pi))

            current_time = time.perf_counter_ns()

            time_diff = current_time - self.last_time
            # hint: the JitterRNG class has a last_time attribute,
            # and you just computed the current time.

            # Extract the least significant bits of the time difference
            # This is where the true randomness comes from
            lsb = time_diff & 0xFF # Get the lowest 8 bits of time_diff
            # hint: bitwise ops
            jitter_data.append(lsb)

            self.last_time = current_time

        return jitter_data

    def fill_entropy_pool(self):
        """Fill the entropy pool with timing jitter data"""
        # Collect enough jitter samples to fill the pool
        jitter_data = self._collect_timing_jitter(self.pool_size)

        # Add to the entropy pool
        for value in jitter_data:
            self.entropy_pool.append(value)

        return jitter_data

    def get_random_bytes(self, num_bytes=32):
        """Generate random bytes using the entropy pool"""
        # Make sure we have enough entropy
        if len(self.entropy_pool) < self.pool_size:
            self.fill_entropy_pool()

        # Mix the entropy pool using SHA-256
        pool_bytes = bytes(self.entropy_pool)
        mixed_entropy = hashlib.sha256(pool_bytes).digest()

        # Create an output buffer
        result = bytearray()

        # Generate requested number of bytes
        while len(result) < num_bytes:
            # Add more entropy to the pool
            self.fill_entropy_pool()

            # Mix new entropy with previous hash
            pool_bytes = bytes(self.entropy_pool)
            h = hashlib.sha256()
            h.update(mixed_entropy)
            h.update(pool_bytes)
            mixed_entropy = h.digest()

            # Add to result
            result.extend(mixed_entropy)

        # Return only the requested number of bytes
        return bytes(result[:num_bytes])

    def get_random_int(self, min_val=0, max_val=100):
        """Generate a random integer between min_val and max_val (inclusive)"""
        # Calculate how many bytes we need
        range_size = max_val - min_val + 1
        if range_size <= 0:
            raise ValueError("Invalid range")

        # Calculate how many bits we need
        bits_needed = range_size.bit_length()
        bytes_needed = (bits_needed + 7) // 8

        # Get random bytes
        random_bytes = self.get_random_bytes(bytes_needed)

        # Convert bytes to integer
        value = int.from_bytes(random_bytes, byteorder='big')

        # Map to our range
        return min_val + (value % range_size)

    def return_random_samples(self, sample_size=1000):
        """Analyze the randomness of the generator"""
        # Generate samples
        samples = []
        for _ in range(sample_size):
            samples.append(self.get_random_int(0, 255))

        return samples


jitter_rng = JitterRNG()
random_data = jitter_rng.return_random_samples(1000)

In [15]:
def bias_value(val: int, bias: float) -> int:
    """
    Tweaks the first eight bits of the given number, making them 1 with ``bias`` probability,
    """
    for i in range(8):
        should_bias = random() < bias
        if not should_bias:
            continue
        val = val | (1 << i)
    return val


def bias_data(random_data: list[int], bias: float) -> list[int]:
    """
    Biases all the numbers in the given data and returns the biased list.
    """
    return [bias_value(v, bias) for v in random_data]


biased_data = bias_data(random_data, 0.3)

In [16]:
# We try to predict the bits like in QRNGs and PRNGs.

import torch
import torch.nn as nn


class NextNumberPredictor(nn.Module):
    """
    Adapted from: https://medium.com/@gpj/predict-next-number-using-pytorch-47187c1b8e33.
    Modified to predict bits of random numbers instead of next numbers.
    """
    def __init__(self, layer_width, layer_depth):
        super(NextNumberPredictor, self).__init__()
        self.lstm = nn.LSTM(1, layer_width, layer_depth)
        self.fc = nn.Linear(layer_width, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x

def train(model, data, num_epochs):
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters())

    for epoch in range(num_epochs):
        for _input_sequence, _target in data:
            _input_sequence = torch.Tensor(_input_sequence).view(len(_input_sequence), -1)
            _target = torch.Tensor(_target).view(len(_target), -1)

            # Forward pass
            output = model(_input_sequence)
            loss = loss_fn(output, _target)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


for (sequence_type, sequence) in (("JitterRNG", random_data), ("BiasedJitterRNG", biased_data)):
    for i in range(5):
        # These dimensions are adjustable, but this NN works well enough to prove our point.
        model = NextNumberPredictor(40, 1)

        data = [((num & (1 << i)) >> i) for num in sequence[:1000]]
        N_TRAIN = 500
        training_rngs = data[:N_TRAIN]
        test_rngs     = data[N_TRAIN:]

        data = [
            (training_rngs[:100], training_rngs[1:101]),
            (training_rngs[:200], training_rngs[1:201]),
            (training_rngs[:300], training_rngs[1:301]),
            (training_rngs[:400], training_rngs[1:401]),
        ]

        # Train the model
        train(model, data, num_epochs=100)

        # Use the model to make predictions
        input_sequence = torch.Tensor(test_rngs[:100]).view(-1, 1)
        output = model(input_sequence)
        predicted_bits = ([0 if x < 0.5 else 1 for x in torch.flatten(output).tolist()])
        actual_bits    = (test_rngs[1:101])

        print(f"Predicting bit#{i} of the {sequence_type} sequence: "
              f"{sum(pred == actual for pred, actual in zip(predicted_bits, actual_bits))}% match.")

Predicting bit#0 of the JitterRNG sequence: 45% match.
Predicting bit#1 of the JitterRNG sequence: 50% match.
Predicting bit#2 of the JitterRNG sequence: 58% match.
Predicting bit#3 of the JitterRNG sequence: 45% match.
Predicting bit#4 of the JitterRNG sequence: 52% match.
Predicting bit#0 of the BaisedJitterRNG sequence: 65% match.
Predicting bit#1 of the BaisedJitterRNG sequence: 61% match.
Predicting bit#2 of the BaisedJitterRNG sequence: 62% match.
Predicting bit#3 of the BaisedJitterRNG sequence: 60% match.
Predicting bit#4 of the BaisedJitterRNG sequence: 65% match.
